In [200]:
import pandas as pd
import sklearn
import datetime
import numpy as np


import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.cluster import DBSCAN

import pandas as pd

from sklearn.tree import DecisionTreeClassifier

In [76]:
df = pd.read_csv('gs://shakingshamrocks_eu/test_data_3_sec.csv')

In [77]:
df['datetime'] = df['dates'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [78]:
def convert_to_mins(val):
    try:
        return(int((val.total_seconds()/60)))
    except ValueError:
        return np.nan

In [79]:
df['time_since_last_transaction'] = df['datetime'] - df['datetime'].shift()
df['time_since_last_transaction']= df['time_since_last_transaction'].apply(lambda x: convert_to_mins(x))

In [80]:
df = df.reset_index()

In [81]:
#time since last transaction in 'other party'
main_dict = {}
for party in list(df['other_account_name'].value_counts().index):
    temp_df = df[df['other_account_name'] == party].copy()
    temp_df['time_since_last_trans_party'] = temp_df['datetime'] - temp_df['datetime'].shift()
    pos_dict = pd.Series(temp_df['time_since_last_trans_party'].values, index = temp_df['index'])
    main_dict.update(pos_dict)
df['time_since_last_transaction_party'] = df['index'].map(main_dict)
df['time_since_last_transaction_party']= df['time_since_last_transaction_party'].apply(lambda x: convert_to_mins(x))

## Clustering

In [84]:
df_test = df[df['account_name'] == 'Michele Powell']
df_test['day'] = pd.to_datetime(df_test.dates).dt.day
df_test.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,Unnamed: 0,dates,account_name,account_IBAN,other_account_name,Type,amount,balance,datetime,time_since_last_transaction,time_since_last_transaction_party,day
1,1,1,2018-02-20 09:39:57,Michele Powell,DWWX82426529885227,Wages MyOrg,Regular,2600.000000,4900.000000,2018-02-20 09:39:57,7135.0,7135.0,20
3,3,3,2018-02-28 04:12:07,Michele Powell,DWWX82426529885227,Medical Subscription,Regular,-150.000000,4530.358998,2018-02-28 04:12:07,3782.0,NaN,28
5,5,5,2018-02-28 07:50:49,Michele Powell,DWWX82426529885227,Food Reg,Irregular,-296.297669,4154.061329,2018-02-28 07:50:49,37.0,4000.0,28
7,7,7,2018-02-28 08:05:12,Michele Powell,DWWX82426529885227,Car,Regular,-250.000000,3704.061329,2018-02-28 08:05:12,0.0,NaN,28
8,8,8,2018-02-28 08:19:49,Michele Powell,DWWX82426529885227,Medical Session,Regular,-200.000000,3504.061329,2018-02-28 08:19:49,14.0,14.0,28


In [203]:
numeric_features = ['amount', 'day', "time_since_last_transaction", "time_since_last_transaction_party"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scaler', StandardScaler())])
# account name used as dummy feature
categorical_features = ['account_name']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')


In [204]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('cluster', DBSCAN(1, min_samples = 5))])

In [205]:
prediction = clf.fit_predict(df_test)

df_test['prediction'] = prediction

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [207]:
df_test[df_test['prediction'] == -1]

,index,Unnamed: 0,dates,account_name,account_IBAN,other_account_name,Type,amount,balance,datetime,time_since_last_transaction,time_since_last_transaction_party,day,prediction
205,205,205,2018-08-28 00:11:15,Michele Powell,DWWX82426529885227,and Sons,Discretionary,-148.194015,-8140.528489,2018-08-28 00:11:15,10833.0,162727.0,28,-1
314,314,314,2019-03-01 04:11:55,Michele Powell,DWWX82426529885227,Medical Subscription,Regular,-150.000000,-8780.175293,2019-03-01 04:11:55,193.0,44731.0,1,-1
316,316,316,2019-03-01 04:55:30,Michele Powell,DWWX82426529885227,Rent - apt 12,Regular,-1500.000000,-10330.175293,2019-03-01 04:55:30,24.0,864.0,1,-1
488,488,488,2020-10-31 05:58:47,Michele Powell,DWWX82426529885227,Utilities,Regular,-208.977165,-5395.300595,2020-10-31 05:58:47,86721.0,86721.0,31,-1
497,497,497,2021-04-30 08:51:33,Michele Powell,DWWX82426529885227,Utilities,Regular,-295.148714,-7494.116366,2021-04-30 08:51:33,87245.0,87308.0,30,-1
509,509,509,2021-12-31 00:42:47,Michele Powell,DWWX82426529885227,Utilities,Regular,-226.285692,-10298.037312,2021-12-31 00:42:47,86303.0,86303.0,31,-1


In [199]:
### Tree Model 

In [210]:
numeric_features = ['amount', 'day', "time_since_last_transaction", "time_since_last_transaction_party"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scaler', StandardScaler())])
# account name used as dummy feature
categorical_features = ['account_name']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [211]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [212]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('cluster', DecisionTreeClassifier())])

In [ ]:
fitted2 = clf2.fit(X_train,y_train)

In [208]:
clf2 = DecisionTreeClassifier()

In [209]:
fitted2 = clf2.fit(X_train,y_train)

NameError: name 'X_train' is not defined